In [2]:
import time
from datetime import datetime 
from urllib.request import urlopen
from bs4 import BeautifulSoup
from pytz import timezone
import pyowm
import numpy as np
import pandas as pd

In [3]:
class Openwinterface():
    def __init__(self, idioma, cidade, pais, chave):
 
        #retorna um dicionário com os dados do openweather
         
        self.retorno = { }
        self.idioma = idioma
        self.cidade = cidade
        self.pais = pais
        self.chave = chave
 
        # só continua se a chave for uma chave válida
        if self.chave[0] == 'x':
            return
 
        self.get_weather()
 
 
    def get_weather(self):
 
        owm = pyowm.OWM(self.chave, language=self.idioma )   # aqui criamos a nossa interface com a biblioteca
         
        posicao = "%s,%s" % (self.cidade, self.pais)        # definimos o local aonde serão obtidos os dados
         
        observation = owm.weather_at_place(posicao)
         
        dados_clima = observation.get_weather()           # lemos todos os dados disponíveis
         
        # aqui convertemos os dados em um dicionario para retornamos todos os dados disponíveis.           
 
        self.retorno['temperatura']  = dados_clima.get_temperature('celsius')
        self.retorno['umidade'] = dados_clima.get_humidity()
        self.retorno['chuva'] = dados_clima.get_rain()
        self.retorno['clima detalhado'] = dados_clima.get_detailed_status()
        self.retorno['clima'] = dados_clima.get_status()
        self.retorno['nuvens'] = dados_clima.get_clouds()
        self.retorno['data'] = dados_clima.get_reference_time(timeformat='iso')
        return self.retorno

In [4]:
weekday_name = ["DOM", "SEGUNDA", "TERÇA", "QUARTA", "QUINTA", "SEXTA", "SAB"]
def hora(retorna = None, weekday_name = weekday_name):
    h = datetime.now().astimezone(timezone('America/Sao_Paulo'))
    name_day = weekday_name[int(h.strftime("%w"))] #Tirar o 5
    h = str(h)
    if retorna == 'd':
        return h[8:10]+"/"+h[5:7]+" "+name_day+"-FEIRA"
    elif retorna == 'da':
        return h[8:10]+"/"+h[5:7]+"/"+h[2:4]
    elif retorna == 'h':
        return h[11:16]
    else:
        return h[11:16]+" "+h[8:10]+"/"+h[5:7]+"/"+h[2:4]+" "+name_day+"-FEIRA"

In [5]:
def pegar_cardapio(retorno = 'vetor',RU = "ru-centro-politecnico"):
    html = urlopen("http://www.pra.ufpr.br/portal/ru/"+RU)
    res = BeautifulSoup(html.read(),"html.parser")
    vetor = res.get_text().split('\n')
    hoje = hora('d')
    vetor = vetor[vetor.index(hoje+"Â\xa0"): vetor.index('* CardÃ¡pio sujeito a alteraÃ§Ãµes devido a questÃµes operacionais e de abastecimento.')]

    for k in range(len(vetor[2:])):
        if "CAFÃ‰ DA MANHÃƒ" in vetor[k+2]:
            vetor = vetor[:k+1]
            break
    vetor[0] = vetor[0][:5]
    del vetor[1], vetor[4], vetor[9]
    texto = ""
    for k in vetor:
        texto += k+"\n"
    texto = texto.replace('Ã‰','É').replace('ƒ','').replace('Ã©','é').replace('Ã¡','á').replace('Ã£','ã').replace('Â','').replace('Ã‡','Ç').replace('Ãª','ê').replace('Ã§','ç').replace('Ã³','ó').replace('Ãµ','õ').replace('\xa0','').replace('Opção vegana:','').replace('Ã\xad','í').replace('Ãº','ú')
    if retorno == 'vetor':
        return texto.split("\n")
    return texto

In [6]:
def salvar_cardapio():
    cardapio = pd.read_csv('cardapio.csv')
    hoje = pegar_cardapio()[:13]
    if (hoje[0] in cardapio['Dia']) == False:
        hojee = pd.DataFrame([hoje], index = [hoje[0]] ,columns = 'Dia,Bebida_c,Pao_c,Fruta_c,Salada_a,Carne_a,Acompanhamento_a,Sobremesa_a,Vegano_a,Salada_j,Carne_j,Acompanhamento_j,Vegano_j'.split(','))
        cardapio = pd.concat([cardapio,hojee],sort = False)
    cardapio.to_csv('cardapio.csv',index = False)

In [7]:
def gravar_tempo():
    clima = pd.read_csv('clima.csv')
    op = Openwinterface("pt","curitiba","Brazil","081ee8d692a1479b7f3edfc5862b3d2a")
    dic = op.get_weather()
    hr = hora()
    tempo = [hr]
    for k in dic:
        if k == 'temperatura':
            for j in dic['temperatura']:
                if j != 'temp_kf':
                    tempo.append(dic['temperatura'][j])
        else:
            tempo.append(dic[k])
    tem = pd.DataFrame([tempo], index = [[hr]] ,columns = 'Dia,Temperatura,Temp_max,Tem_min,Umidade,Chuva,Clima_detalhado,Clima,Nuvens,Data_verificação'.split(','))
    clima = pd.concat([clima,tem],sort = False)
    clima.to_csv('clima.csv',index = False)

In [8]:
gravar_tempo()
print('Tempo salvo',hora())
salvar_cardapio()
print('Cardápio salvo',hora())
print('Terminou')

Tempo salvo 10:18 13/09/19 SEXTA-FEIRA
Cardápio salvo 10:18 13/09/19 SEXTA-FEIRA
Terminou


In [10]:
c = pd.read_csv('cardapio.csv')
t = pd.read_csv('clima.csv')

In [11]:
c.head()

,Dia,Bebida_c,Pao_c,Fruta_c,Salada_a,Carne_a,Acompanhamento_a,Sobremesa_a,Vegano_a,Salada_j,Carne_j,Acompanhamento_j,Vegano_j
0,09/09,Café/leite/chá,Pão compatê de azeitona,Fruta,Acelga e beterraba ralada,"Moída mexicana (molho de tomate, molho de pime...",Macarrão alho e óleo,Fruta,chilli de PTS,Escarola e rabanete,Copa lombo ao molho oriental,"Sopa londrina (carne bovina, batata, cenoura, ...",charuto de repolho (grão de bico)
1,11/09,Café/leite/chá,Pão com queijo,Fruta,Almeirão e abobrinha ralada,Frango crocante (farinha de rosca e queijo par...,Quibebe (bacon),Flan de baunilha,bolinho de feijão cavalo (sem glúten),Couve e natural,"Bife a pizzaiolo (molho de tomate, presunto e ...","Sopa de casa (frango, feijão branco, abobrinha...",cebola recheada com lentilha
2,12/09,Café/leite/chá,Pão compatê de presunto,Fruta,Alface crespa e rabanete,Bife de panela (molho de tomate),Bolinho de arroz,Mamão fatiado,lentilha ao sugo,Radite e beterraba ralada,Frango ao molho branco,"Sopa eslava (carne bovina, batata, cenoura e m...",hambúrguer de ervilha seca (s/ glúten)
3,13/09,Café/leite/chá,Pão comdoce de abacaxi,Fruta,Radite e cenoura ralada,Filé de frango ao m. açafrão (molho branco com...,"Ratatouille (berinjela, abobrinha, acelga, tom...",Gelatina,hambúrguer de feijão branco,Mix de alface e tomate,Posta ao molho sugo,"Sopa de raízes (aipim, batata, batata salsa, c...","espetinho de bolinha de feijão cavalo, com pi..."


In [12]:
t.head()

,Dia,Temperatura,Temp_max,Tem_min,Umidade,Chuva,Clima_detalhado,Clima,Nuvens,Data_verificacao,Data_verificação
0,09:50 11/09/19 QUARTA-FEIRA,24.03,26.67,20.0,68,{},céu limpo,Clear,0,NaN,2019-09-11 12:49:50+00
1,09:50 11/09/19 QUARTA-FEIRA,24.03,26.67,20.0,68,{},céu limpo,Clear,0,NaN,2019-09-11 12:49:50+00
2,09:56 11/09/19 QUARTA-FEIRA,24.03,26.67,20.0,68,{},céu limpo,Clear,0,NaN,2019-09-11 12:49:50+00
3,09:57 11/09/19 QUARTA-FEIRA,24.03,26.67,20.0,68,{},céu limpo,Clear,0,NaN,2019-09-11 12:49:50+00
4,23:16 12/09/19 QUINTA-FEIRA,18.52,19.00,18.0,88,{},nuvens dispersas,Clouds,40,NaN,2019-09-13 02:12:33+00


In [27]:
#https://pyowm.readthedocs.io/en/latest/usage-examples-v2/weather-api-usage-examples.html#create-global-owm-object
# ctrl + f (A Forecast object encapsulates the Location object relative to the forecast and a list of Weather objects)

datetime.datetime(2013, 9, 13, 17, 0)